Import Classes and Define Variables

In [1]:
import numpy as np
import scipy as sp
import pymongo
import pandas as pd
import nsepy
from datetime import datetime
from datetime import timedelta
from pymongo import MongoClient
from nsepy import get_history
client=MongoClient("mongodb://localhost:27018/",username="stockadm",password="1qaz@WSX")
db=client["stockWarehouse"]
equityList=db["stocklist"]
stockWarehouse=db["stockWarehouse"]
equityListPd = pd.DataFrame(equityList.find({}))
equityListPd.shape[0]

8426

Data Collector Function

In [2]:
def dataCollector():
    equityListPd = pd.DataFrame(equityList.find({'Security Name':'Active','State':"A"}))
    collectionTime=2000
    basicSample=30
    cnt=0
    my_time = datetime.now()

    for rows in equityListPd['Issuer Name'].iteritems():
        print(cnt,rows[1],end="-")
        cnt= cnt + 1
        if stockWarehouse.find({'Symbol':rows[1]}).count() > 1 :
            for j in stockWarehouse.find({'Symbol':rows[1]},{'_id':0,'Date':1}).sort('Date',-1).limit(1) :
                collectionTime=((my_time - j['Date']).days - 1)
                fromDate=j['Date']
        iters=int(collectionTime/basicSample)
        if (collectionTime != 0):
            if collectionTime < 2000:
                basicSample=collectionTime
                iters=1
            dt=datetime.now()
            print("Iterations:",iters,end="|")
            for i in range(0,iters):

                startDate=dt - timedelta(days=basicSample)
                if( startDate <= fromDate):
                    startDate = (fromDate + timedelta(days=1))
                endDate=dt 
                print(i,"Collecting into between",startDate.date(),"-", endDate.date(),"@",datetime.now(),end="|")
                df=pd.DataFrame(get_history(symbol=rows[1] , start= startDate,end=endDate))
                df.reset_index(inplace=True)
                df['Date']=pd.to_datetime(df['Date'])
                dfLength=(len(df.index))
                if  dfLength > 0 :
                    data_dict=df.to_dict("records")
                    stockWarehouse.insert_many(data_dict)
                    tmpdf=pd.DataFrame(stockWarehouse.find({"Symbol":rows[i],"Date":{"$lte":endDate,"$gte":startDate}}))
                    print(df.index.size,tmpdf.index.size,end="|")

                else :
                    print("Marking as Inactive")
                    equityList.update_many({'Issuer Name':rows[1]},{'$set': {'Security Name':'Inactive'}})
                    break;
                dt=(startDate - timedelta(days=1))


In [3]:
def markInactiveStocks():
    reference=int(stockWarehouse.find({'Symbol':'SBIN'}).count()) - 100
    print(reference)
    tmp_count=0
    for i in equityList.find({},{'Issuer Name':1}):
        #print(i['Issuer Name'])
        tmp_count=(int(stockWarehouse.find({'Symbol':i['Issuer Name']}).count()))
        if tmp_count >= reference:
            print(i['Issuer Name'],end='|')
            equityList.update_many({'Issuer Name':i['Issuer Name']},{'$set': {'State':"A"}})

In [4]:
def markTrend_Crossover(stockdf,datadf,symbol):
    
    for i in range(0,stockdf.shape[0],1):
            if stockdf['Issuer Name'].iloc[i]==symbol:
                break;
    datadf['ewmdiff']=datadf['Close'].ewm(span=25,adjust=False).mean() - datadf['Close'].ewm(span=40,adjust=False).mean()
    nrows=datadf.shape[0]-1
    if datadf['ewmdiff'].iloc[nrows] > 0:
        stockdf['trend'].iloc[i]=1
    else:
        stockdf['trend'].iloc[i]=-1
    for j in range(datadf.shape[0]-1,2,-1):
        if datadf['ewmdiff'].iloc[j]*datadf['ewmdiff'].iloc[j-1] <= 0:
            stockdf['daysSinceCutOver'].iloc[i]=datadf.shape[0]-j            
            break;
    if datadf['ewmdiff'].iloc[nrows] > datadf['ewmdiff'].iloc[nrows-1] and datadf['ewmdiff'].iloc[nrows-1] > datadf['ewmdiff'].iloc[nrows-2]:
        stockdf['trendnature'].iloc[i]='Expanding'
    else:
        stockdf['trendnature'].iloc[i]='Contracting'
    return(stockdf)

In [5]:
def singlePatternCandle(stockdf,datadf,symbol):
    for i in range(0,stockdf.shape[0],1):
        if stockdf['Issuer Name'].iloc[i]==symbol:
            break;
                
    pattern=''
    umbrellaFound=False
    if 'singleCandlePattern' in stockdf.columns:
        if pd.isnull(stockdf['singleCandlePattern'].iloc[i]):
            pattern=''
        else:
            pattern=stockdf['singleCandlePattern'].iloc[i]
    nrows=datadf.shape[0]-1
    Close=datadf['Close'].iloc[nrows]
    Open=datadf['Open'].iloc[nrows]
    High=datadf['High'].iloc[nrows]
    Low=datadf['Low'].iloc[nrows]
    if  Open > Close :
        candle=-1
    if  Open < Close :
        candle=1
    if  Open == Close :
        candle=0
####### Check MaruBozo
    
    if candle > 0:
        if ((High-Close)/(Close-Open)) < .05 and ((Open-Low)/(Close-Open))< .05:
            pattern=pattern + 'Bull Marubozo|' +'SL:' + str(Low)
    if candle < 0:
        if ((High-Open)/(Open-Close)) < .05 and ((Close-Low)/(Open-Close)) < .05:
            pattern=pattern + 'Bear Marubozo|' +'SL:' + str(High)
        
####### Check Paper Umbrella
    if candle > 0:
        if ((Open-Low)/(Close-Open)) > 2 and (High-Close)/(Close-Open) < .05:
            umbrellaFound=True
    if candle < 0:
        if ((Close-Low)/(Open-Close)) > 2 and (High-Open)/(Open-Close) < .05:
            umbrellaFound=True
    if umbrellaFound:
        if stockdf['trend'].iloc[i] == 1:
            pattern=pattern + 'Hanging Man ' +'SL:' + str(High)
        else:
            pattern=pattern + 'Hammer ' + 'SL:' + str(Low)
        
    stockdf['singleCandlePattern'].iloc[i]=pattern
    return(stockdf)
                
        
        

In [6]:
#dataCollector()

In [7]:
#markInactiveStocks()

In [8]:
equityList=db["stocklist"]
stockWarehouse=db["stockWarehouse"]

equityListdf=pd.DataFrame(equityList.find({'Security Name':'Active','State':"A"},{'Issuer Name':1,'_id':0}))
equityListdf['trend']=''
equityListdf['daysSinceCutOver']=''
equityListdf['trendnature']=''
equityListdf['singleCandlePattern']=''
       
for dt in stockWarehouse.find({'Symbol':'SBIN'},{'_id':0,'Date':1}).sort('Date',-1).limit(1):
    refdate=dt # Set Reference Data as last date when SBI Stock Data was update
print(refdate)
i=0
for symbol in equityListdf['Issuer Name']    :
    for maxdate in stockWarehouse.find({'Symbol':symbol},{'_id':0,'Date':1}).sort('Date',-1).limit(1):
        if maxdate == refdate:  # Check if database has latest sttock info by comparing aginst SBI
            datadf=pd.DataFrame(stockWarehouse.find({'Symbol':symbol}).sort('Date',-1).limit(300))# Get top 300 row of stock in iteration
            datadf.set_index('Date',inplace=True)
            datadf.sort_index(inplace=True)
            equityListdf=markTrend_Crossover(equityListdf,datadf,symbol)
            equityListdf=singlePatternCandle(equityListdf,datadf,symbol)
            
            
print(equityListdf)
equityListdf.to_csv("C:\\Users\\khanna's\\Documents\\Stock Trend Analysis\\result.csv")

{'Date': datetime.datetime(2020, 7, 17, 0, 0)}
    Issuer Name trend daysSinceCutOver  trendnature      singleCandlePattern
0           ABB     1               12  Contracting                         
1    AMARAJABAT     1               38  Contracting                         
2          HDFC     1               22  Contracting                         
3    ASIANHOTNR    -1              292  Contracting                         
4          ATUL     1               36    Expanding                         
5    AUTOLITIND    -1                7  Contracting                         
6     BAJAJHIND     1               33  Contracting                         
7    BAJFINANCE     1               19    Expanding                         
8    BANCOINDIA     1               19  Contracting                         
9    CENTURYTEX    -1               90  Contracting                         
10    BATAINDIA    -1               22    Expanding                         
11    BANARISUG     1        